In [2]:
import numpy as np

In [5]:
class DecisionTree:
    def __init__(self, depth=5, min_leaf_size=5):
        self.depth = depth
        self.decision_boundary = 0
        self.left = None
        self.right = None
        self.min_leaf_size = min_leaf_size
        self.prediction = None

    def mean_squared_error(self, labels, prediction):
        if labels.ndim != 1:
            print("Error: Input labels must be one dimensional")

        return np.mean((labels-prediction)**2)
    
    def train(self, x, y):
        if x.ndim != 1:
            raise ValueError("Input dataset must be one dimensional")
        if len(x) != len(y):
            raise ValueError("x and y have different lengths")
        if y.ndim != 1:
            raise ValueError("Dataset labels must be one dimensional")
        if len(x) < 2*self.min_leaf_size:
            self.prediction = np.mean(y)
            return
        if self.depth == 1:
            self.prediction = np.mean(y)
            return
        best_split = 0
        min_error = self.mean_squared_error(x, np.mean(y))*2

        for i in range(len(x)):
            if len(x[:i]) < self.min_leaf_size:
                continue
            elif len(x[i:]) < self.min_leaf_size:
                continue
            else:
                error_left = self.mean_squared_error(x[:i], np.mean(y[:i]))
                error_right = self.mean_squared_error(x[i:], np.mean(y[i:]))
                error = error_left + error_right
                if error < min_error:
                    best_split = i
                    min_error = error
        if best_split != 0:
            left_x = x[:best_split]
            left_y = y[:best_split]
            right_x = x[best_split:]
            right_y = y[best_split:]

            self.decision_boundary = x[best_split]
            self.left = DecisionTree(
                depth=self.depth-1, min_leaf_size=self.min_leaf_size
            )
            self.right = DecisionTree(
                depth=self.depth-1, min_leaf_size=self.min_leaf_size
            )
            self.left.train(left_x, left_y)
            self.right.train(right_x, right_y)
        else:
            self.prediction = np.mean(y)
        return
    
    def predict(self, x):
        if self.prediction is not None:
            return self.prediction
        elif self.left or self.right is not None:
            if x >= self.decision_boundary:
                return self.right.predict(x)
            else:
                return self.left.predict(x)
        else:
            print("Error: Decision Tree not yet trained")
            return None
        
class TestDecisionTree:
    def helper_mean_squared_error_test(labels, prediction):
        squared_error_sum = float(0)
        for label in labels:
            squared_error_sum += (label-prediction)**2

        return float(squared_error_sum / labels.size)
    
def main():
    x = np.arange(-1.0, 1.0, 0.005)
    y = np.sin(x)
    tree = DecisionTree(depth=10, min_leaf_size=10)
    tree.train(x, y)

    rng = np.random.default_rng()
    test_cases = (rng.random(10)*2) - 1
    predictions = np.array([tree.predict(x) for x in test_cases])
    avg_error = np.mean((predictions-test_cases)**2)

    print("test values: " + str(test_cases))
    print("predictions: " + str(predictions))
    print("average error: " + str(avg_error))

if __name__ == "__main__":
    main()


test values: [-0.51970176  0.90821816 -0.01575481  0.82869334  0.50568716  0.97766214
  0.94456177  0.18559701  0.50701883  0.77187148]
predictions: [-0.47273421  0.7940027  -0.02999175  0.72936277  0.46832345  0.82478261
  0.7940027   0.19128482  0.46832345  0.69426669]
average error: 0.008030879623547958
